# 请求体参数
参数有三种类型：路径参数、查询参数和请求体参数。

如果一个参数是url路径中的参数，那么这个参数就会被默认是路径参数。

如果一个单个参数不是url路径中的参数，那么这个参数就会被默认是查询参数。即便是用post方法，参数也会被默认是查询参数。

如果一个参数的类型是一个**pydantic模型**，那么这个参数就会被默认是请求体参数。

In [ ]:
# 请求体参数必须用 Body 声明，否则会被当作查询参数处理
from fastapi import FastAPI, Body
from typing import Annotated
import uvicorn
app = FastAPI()
@app.post("/items/")
async def create_item(item: Annotated[int,Body()]):
    return {"item": item}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

"""
请求体意味着用json格式发送数据，而不是直接在url里拼接
import requests

url = "http://127.0.0.1:8009/items/"
response = requests.post(url,json=5)
print(response.text)

输出 {"item":5}
"""

INFO:     Started server process [25208]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:53628 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53629 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:53647 - "POST /items/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:53749 - "GET /items/5 HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53749 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53750 - "GET /items?5 HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:53750 - "GET /items/?5 HTTP/1.1" 405 Method Not Allowed
INFO:     127.0.0.1:58792 - "GET /items/?item=5 HTTP/1.1" 405 Method Not Allowed


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [25208]


#### 多个请求体 Body()
差别在传入时，可以用字典定义多个要传入的请求体

In [ ]:
from fastapi import FastAPI, Body
from typing import Annotated
import uvicorn
app = FastAPI()
@app.post("/items/")
async def get_data(
    item: Annotated[int, Body()],
    name: Annotated[str, Body()],
):
    return {"item": item, "name": name}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8009)
    server = uvicorn.Server(config)
    await server.serve()
"""
import requests
url = 
data = {"item": 5,
        "name": "foo"}           用字典定义多个请求信息
response = requests.post(url,json=data)
"""

#### Pydantic模型参数
用于校验对应类型的参数是否符合自定义的要求

同样的，也需要在requests里传入字典

In [ ]:
from fastapi import FastAPI
import uvicorn
from pydantic import BaseModel
app = FastAPI()

class Item(BaseModel):
    name: str
    description: Union[str, None] = None
    price: float
    tax: Union[float, None] = None

@app.post("/items/")
async def create_item(item: Item):
    return item

""""
url = 
data = {
    "name": "Foo",
    "description": "A very nice Item",
    "price": 35.4,
    "tax": 3.2,
}
response = requests.post(url,json=data)
print(response.text)
"""

#### 多个Pydantic 
fastapi 会自动用记住对应参数的名字

传入的时候构建字典，用对应参数名字作为键

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

app = FastAPI()
class Item(BaseModel):
    name: str
    description: Union[str, None] = None
    price: float
    tax: Union[float, None] = None

class User(BaseModel):
    username: str
    full_name: Union[str, None] = None

@app.post("/items/")
async def create_item(item: Item, user: User):
    return {"item": item, "user": user}
if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8000)
    server = uvicorn.Server(config)
    await server.run()

"""
url = 
data={
    "item": {
        "name": "Foo",
        "description": "A very nice Item",
        "price": 35.4,
        "tax": 3.2,
    },
    "user": {
        "username": "johndoe",
        "full_name": "John Doe",
    },
}
"""

#### 嵌入单个请求体参数 Body()+pydantic
对于多个pydantic，fastapi会记住参数名字，用户可以将参数名字作为键来构建data字典

但是对于单个pydantic，可以使用 Body(embed=True), 使得单个pydantic的情况下，用户也可以使用参数名字作为键来构建data字典。

In [ ]:
from fastapi import FastAPI, Body
from pydantic import BaseModel
import uvicorn
app = FastAPI()
class Item(BaseModel):
    name: str
    description: Union[str, None] = None
    price: float
    tax: Union[float, None] = None
@app.post("/items/")
async def create_item(item: Item = Body(embed=True)): # 使用embed=True 
    return item

"""
url =
data = {
    "item": {               使用embed=True , 即使是单个pydantic模型也可以使用参数名来作为键
        "name": "Foo",
        "description": "A very nice Item",
        "price": 35.4,
        "tax": 3.2,
    },
}
"""

# 额外参数信息 Field
类似 Query、Path 和 Body ，都是为了增加元数据或者校验。不过这些是作用在路径操作函数的

而Field是作用在pydantic模型里的

可以在 Field、Query、Path、Body 中声明额外的信息。这些信息将包含在生成的 JSON Schema 中。

In [ ]:
from pydantic import BaseModel, Field
from fastapi import FastAPI
import uvicorn
app = FastAPI()
class Item(BaseModel):  # 使用Field 来声明额外的元数据（如title）或者校验（max_length、gt等）
    name: str = Field(..., title="The name of the item", max_length=300)

    description: Union[str, None] = Field(
        None, title="The description of the item", max_length=300
    )

    price: float = Field(..., gt=0, description="The price must be greater than zero")
    tax: Union[float, None] = None
@app.post("/items/")
async def create_item(item: Item):
    return item
if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8000)
    server = uvicorn.Server(config)
    await server.run()

"""
请求方式一样
"""

#### 额外向型--文档中提供示例
1. 通过在pydantic定义模型时使用 model_config 和 json_schema_extra 字段来定义一个example，fastapi会读取字段内容，并且自动 在 文档 中为用户提供example

2. 通过在Field里使用 example参数，也有同样效果

3. 同样的，Body，Query，Path参数也有example参数

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel,Field
import uvicorn
app = FastAPI()
class Item(BaseModel):
    name: str = Field(..., example="Foo")   # 通过在Field里使用 example参数，来为文档提供示例
    description: Union[str, None] = None
    price: float
    tax: Union[float, None] = None
    # 这里的model config不是让用户传入数据，而是在告诉fastapi要在文档里生成一个如下的示例
    model_config = {
        "json_schema_extra": {
            "example": {
                "name": "Foo",
                "description": "A very nice Item",
                "price": 35.4,
                "tax": 3.2,
            }
        }
    }
class User(BaseModel):
    username: str = Field(..., example="johndoe")
    full_name: Union[str, None] = Field(None, example="John Doe")

# 用于传入Body的example
user_example = {
    "username": "johndoe",
    "full_name": "John Doe",
}

@app.post("/items/")
async def create_item(item: Item, user: User = Body(example = user_example)): # 通过在Body里使用 example参数，也可以为文档提供示例，这里传入了一个字典
    return {"item": item, "user": user}
if __name__ == '__main__':  
    config = uvicorn.Config(app, host='127.0.0.1', port=8000)   
    server = uvicorn.Server(config) 
    await server.run()
"""
请求方式一样, 不需要model config
data = {
    "name": "Foo",  
    "description": "A very nice Item",
    "price": 35.4,  
    "tax": 3.2,
}
"""

#### 额外数据类型
除了常见的 str，int，float，还有一些数据类型在实践中很常用

1. UUID： 一种标准的 "通用唯一标识符" ，在许多数据库和系统中用作ID。 在请求和响应中将以 str 表示
2. datatime.datatime: 在请求和响应中将表示为 ISO 8601 格式的 str ，比如: 2008-09-15T15:53:00+05:00.
3. datetime.date: 在请求和响应中将表示为 ISO 8601 格式的 str ，比如: 2008-09-15. 
4. datetime.time: 在请求和响应中将表示为 ISO 8601 格式的 str ，比如: 14:23:55.003
5. datetime.timedelta: 在请求和响应中将表示为 float 代表总秒数。

In [ ]:
import uvicorn
from datetime import datetime, time, timedelta
from typing import Annotated
from uuid import UUID
from fastapi import Body, FastAPI

app = FastAPI()

@app.post("/items/{item_id}")
async def read_items(
    item_id: UUID,
    start_datetime: Annotated[datetime | None, Body()] = None,
    end_datetime: Annotated[datetime | None, Body()] = None,
    repeat_at: Annotated[time | None, Body()] = None,
    process_after: Annotated[timedelta | None, Body()] = None,
):
    start_process = start_datetime + process_after
    duration = end_datetime - start_process
    return {
        "item_id": item_id,
        "start_datetime": start_datetime,
        "end_datetime": end_datetime,
        "repeat_at": repeat_at,
        "process_after": process_after,
        "start_process": start_process,
        "duration": duration,
    }

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()
"""
url
uuid = uuid.uuid4()

data = {
    "item_id": str(uuid),   # UUID 需要转换成字符串
  "start_datetime": "2024-01-24 07:42:54",
  "end_datetime": "2024-01-24 07:43:54",
  "repeat_at": "07:41:54",
  "process_after": 23.567
}
res = requests.post(url, json=data) 
res.text
print(res.text)
"""

# 嵌套模型
pydantic模型里需要类型，但是pydantic模型本身也是一个类型。

因此对于 pydantic模型里的某个字段，使该字段的类型为了一个pydantic模型

In [ ]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()


class Image(BaseModel):
    url: str
    name: str


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None
    image: Image | None = None # image的类型可以是None，也可是一个在上面定义的pydantic模型

# ......... 其它方法一样

"""
传入时要注意 image 的值
data = {
    "name": "Foo",
    "description": "A very nice Item",
    "price": 35.4,
    "tax": 3.2,
    "image": {
        "url": "http://example.com/image.png",
        "name": "example image"
    }
}   
"""

#### 特殊类型
有时候希望检验url字段的内容是不是一个url链接，而不仅仅校验是不是 str

或者希望校验 email 字段的内容是不是一个邮箱

这时候可以用一些python提供的特殊类：HttpUrl （邮箱也有一个对应的类）

In [ ]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel, HttpUrl

app = FastAPI()


class Image(BaseModel):
    url: HttpUrl  # 使用 HttpUrl 来校验url字段； 这样如果传入 url = "aaa" 就是错的
    name: str


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None
    images: list[Image] | None = None  # 这其实又嵌套了一层类型，这时候可以接受多个Image对象，[Image1, Image2, ...]

"""
data = {
    "name": "Foo",
    "description": "The pretender",
    "price": 42.0,
    "tax": 3.2,
    "images": [
        {
            "url": "http://example.com/baz.jpg",
            "name": "The Foo live"
        },
        {
            "url": "http://example.com/dave.jpg",
            "name": "The Baz"
        }
    ]
}
"""

# 任意dict 请求体
无需事先知道有效的字段/属性（在使用 Pydantic 模型的场景）名称是什么。

In [ ]:
import uvicorn
from fastapi import FastAPI

app = FastAPI()


@app.post("/index-weights/")
async def create_index_weights(weights: dict):  # weight的类型是一个任意的dict，只要传入的是字典就行
    return weights

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

"""
url = 'http://127.0.0.1:8009/index-weights/' 
data = {"item_id": 5, "name": "张三"}
或者  （只要是字典就可以）
data = {
    "name": "Foo",
    "description": "The pretender",
    "price": 42.0,
    "tax": 3.2,
    "images": [
        {
            "url": "http://example.com/baz.jpg",
            "name": "The Foo live"
        },
        {
            "url": "http://example.com/dave.jpg",
            "name": "The Baz"
        }
    ]
}
res = requests.post(url, json=data) 
res.text

"""

# Hearder 参数
用于让用户在请求时添加header定义

In [ ]:
import uvicorn
from typing import Annotated
from fastapi import FastAPI, Header
app = FastAPI()
@app.get("/items/")
async def read_items(user_agent: Annotated[str | None, Header()] = None): # Header 用于声明header参数
    return {"User-Agent": user_agent}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8009)
    server = uvicorn.Server(config)
    await server.serve()
"""
url = "http://127.0.0.1:8009/items/"
headers = {"User-Agent": "my-app/0.0.1"}
response = requests.get(url, headers=headers)
print(response.text)
输出 {"User-Agent": "my-app/0.0.1"}
"""

# 现在常见的传入是 requests.post(url, headers=headers, json=data) 形式